In [ ]:
# Sentiment Analysis Project
import pandas as pd



In [ ]:
train_data = "/Users/mac/SENTIMENT ANALYSIS PROJECT/Dataset/e commerce reviews train.csv"

In [ ]:
test_data = "/Users/mac/SENTIMENT ANALYSIS PROJECT/Dataset/e commerce reviews test.csv"

In [ ]:
train_df = pd.read_csv(train_data)


In [ ]:
test_df = pd.read_csv(test_data)

In [ ]:
test_df

In [ ]:
train_df

In [ ]:
train_df['labels'].unique()

array(['__label__2', '__label__1'], dtype=object)

In [ ]:
#mapping values
mapped_values = {
    "__label__2": "Positive",
    "__label__1": "Negative"

}


In [ ]:
train_df['labels'] = train_df['labels'].map(mapped_values)

In [ ]:
test_df['labels'] = test_df['labels'].map(mapped_values)

In [ ]:
train_df

,labels,text
0,Positive,Stuning even for the non-gamer: This sound tra...
1,Positive,The best soundtrack ever to anything.: I'm rea...
2,Positive,Amazing!: This soundtrack is my favorite music...
3,Positive,Excellent Soundtrack: I truly like this soundt...
4,Positive,"Remember, Pull Your Jaw Off The Floor After He..."
...,...,...
3600005,Negative,Don't do it!!: The high chair looks great when...
3600006,Negative,"Looks nice, low functionality: I have used thi..."
3600007,Negative,"compact, but hard to clean: We have a small ho..."
3600008,Negative,what is it saying?: not sure what this book is...


In [ ]:
test_df

,labels,text
0,Positive,Great CD: My lovely Pat has one of the GREAT v...
1,Positive,One of the best game music soundtracks - for a...
2,Negative,Batteries died within a year ...: I bought thi...
3,Positive,"works fine, but Maha Energy is better: Check o..."
4,Positive,Great for the non-audiophile: Reviewed quite a...
...,...,...
399995,Negative,Unbelievable- In a Bad Way: We bought this Tho...
399996,Negative,"Almost Great, Until it Broke...: My son reciev..."
399997,Negative,Disappointed !!!: I bought this toy for my son...
399998,Positive,Classic Jessica Mitford: This is a compilation...


TEXT PREPROCESSING


In [ ]:
def remove_stopwords(text):
    """
    Remove common English stopwords.
    """
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(filtered_tokens)




In [ ]:
train_df['text'].head(10).apply(remove_stopwords)

0    Stuning even non-gamer : This sound track beau...
1    The best soundtrack ever anything . : I 'm rea...
2    Amazing ! : This soundtrack favorite music tim...
3    Excellent Soundtrack : I truly like soundtrack...
4    Remember , Pull Your Jaw Off The Floor After H...
5    absolute masterpiece : I quite sure actually t...
6    Buyer beware : This self-published book , want...
7    Glorious story : I loved Whisper wicked saints...
8    A FIVE STAR BOOK : I finished reading Whisper ...
9    Whispers Wicked Saints : This easy read book m...
Name: text, dtype: object

In [ ]:
train_df['Stopwords'] = train_df['text'].apply(remove_stopwords)

KeyboardInterrupt: 

In [ ]:
test_df['Stopwords'] = test_df['text'].apply(remove_stopwords)

In [ ]:
#Save new csv
train_df.to_csv('train_dataset_updated',index = False)

In [ ]:
train_df_updated = pd.read_csv('/Users/mac/SENTIMENT ANALYSIS PROJECT/train_dataset_updated')

In [ ]:
train_df_updated

In [ ]:
test_df

,labels,text,Stopwords
0,Positive,Great CD: My lovely Pat has one of the GREAT v...,Great CD : My lovely Pat one GREAT voices gene...
1,Positive,One of the best game music soundtracks - for a...,One best game music soundtracks - game I n't r...
2,Negative,Batteries died within a year ...: I bought thi...,Batteries died within year ... : I bought char...
3,Positive,"works fine, but Maha Energy is better: Check o...","works fine , Maha Energy better : Check Maha E..."
4,Positive,Great for the non-audiophile: Reviewed quite a...,Great non-audiophile : Reviewed quite bit comb...
...,...,...,...
399995,Negative,Unbelievable- In a Bad Way: We bought this Tho...,Unbelievable- In Bad Way : We bought Thomas so...
399996,Negative,"Almost Great, Until it Broke...: My son reciev...","Almost Great , Until Broke ... : My son reciev..."
399997,Negative,Disappointed !!!: I bought this toy for my son...,Disappointed ! ! ! : I bought toy son loves ``...
399998,Positive,Classic Jessica Mitford: This is a compilation...,Classic Jessica Mitford : This compilation wid...


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier, PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Set the path to your training CSV file
csv_file = '/Users/mac/SENTIMENT ANALYSIS PROJECT/train_dataset_updated'

# Define a chunksize that fits your memory capacity
chunksize = 10000


# 1. Fit the TF-IDF Vectorizer on a Sample Chunk

# Read the first chunk to fit the TF-IDF vectorizer
first_chunk = pd.read_csv(csv_file, chunksize=chunksize).__next__()
tfidf_vectorizer = TfidfVectorizer(max_features=50000, min_df=5, max_df=0.8)
tfidf_vectorizer.fit(first_chunk['Stopwords'])
print("TF-IDF vectorizer fitted on the first chunk.")


# 2. Initialize Incremental Learning Models

# Define the list of possible classes (ensure they exactly match the label values)
classes = ['Negative', 'Positive']

# Initialize classifiers that support partial_fit
sgd_clf = SGDClassifier(loss='log_loss')  # Logistic Regression variant via SGD
pa_clf = PassiveAggressiveClassifier()
nb_clf = MultinomialNB()

# 3. Process the Training Data in Chunks

print("Starting incremental training on chunks...")
for chunk in pd.read_csv(csv_file, chunksize=chunksize):
    # Transform the text data using the fitted TF-IDF vectorizer
    X_chunk = tfidf_vectorizer.transform(chunk['Stopwords'])
    y_chunk = chunk['labels']

    # Update each model incrementally
    sgd_clf.partial_fit(X_chunk, y_chunk, classes=classes)
    pa_clf.partial_fit(X_chunk, y_chunk, classes=classes)
    nb_clf.partial_fit(X_chunk, y_chunk, classes=classes)

print("Incremental training complete.")

#
X_test = tfidf_vectorizer.transform(test_df['Stopwords'])
y_test = test_df['labels']

# Get predictions from each model
sgd_preds = sgd_clf.predict(X_test)
pa_preds = pa_clf.predict(X_test)
nb_preds = nb_clf.predict(X_test)

# Print the classification reports
from sklearn.metrics import classification_report, accuracy_score

print("=== SGDClassifier (Logistic Regression via SGD) ===")
print("Accuracy:", accuracy_score(y_test, sgd_preds))
print(classification_report(y_test, sgd_preds))

print("=== PassiveAggressiveClassifier ===")
print("Accuracy:", accuracy_score(y_test, pa_preds))
print(classification_report(y_test, pa_preds))

print("=== MultinomialNB ===")
print("Accuracy:", accuracy_score(y_test, nb_preds))
print(classification_report(y_test, nb_preds))


TF-IDF vectorizer fitted on the first chunk.
Starting incremental training on chunks...
Incremental training complete.
=== SGDClassifier (Logistic Regression via SGD) ===
Accuracy: 0.878205
              precision    recall  f1-score   support

    Negative       0.88      0.88      0.88    200000
    Positive       0.88      0.88      0.88    200000

    accuracy                           0.88    400000
   macro avg       0.88      0.88      0.88    400000
weighted avg       0.88      0.88      0.88    400000

=== PassiveAggressiveClassifier ===
Accuracy: 0.8783275
              precision    recall  f1-score   support

    Negative       0.88      0.88      0.88    200000
    Positive       0.88      0.88      0.88    200000

    accuracy                           0.88    400000
   macro avg       0.88      0.88      0.88    400000
weighted avg       0.88      0.88      0.88    400000

=== MultinomialNB ===
Accuracy: 0.8509575
              precision    recall  f1-score   support

   

In [ ]:
import joblib

# Save the model
joblib.dump(sgd_clf, 'sentiment_model.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')


In [ ]:
import joblib

# Load the saved model and vectorizer
model = joblib.load('sentiment_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')

def predict_sentiment(text):
    # Transform the input text using the TF-IDF vectorizer
    X_input = vectorizer.transform([text])
    # Predict the sentiment
    prediction = model.predict(X_input)[0]
    return prediction

# Test the inference function
sample_text = "i love it"
print("Predicted Sentiment:", predict_sentiment(sample_text))


Predicted Sentiment: Positive


In [ ]:
import pandas as pd
import os

# Specify the path to your large CSV file
input_file = "/Users/mac/SENTIMENT ANALYSIS PROJECT/train_dataset_updated"

# Create an output directory to store the chunks
output_dir = "data_chunks"
os.makedirs(output_dir, exist_ok=True)

# Define the number of rows per chunk (adjust as needed)
chunksize = 100000

# Read the CSV file in chunks and save each chunk to a separate CSV
for i, chunk in enumerate(pd.read_csv(input_file, chunksize=chunksize)):
    output_file = os.path.join(output_dir, f"chunk_{i+1}.csv")
    chunk.to_csv(output_file, index=False)
    print(f"Saved {output_file}")


Saved data_chunks/chunk_1.csv
Saved data_chunks/chunk_2.csv
Saved data_chunks/chunk_3.csv
Saved data_chunks/chunk_4.csv
Saved data_chunks/chunk_5.csv
Saved data_chunks/chunk_6.csv
Saved data_chunks/chunk_7.csv
Saved data_chunks/chunk_8.csv
Saved data_chunks/chunk_9.csv
Saved data_chunks/chunk_10.csv
Saved data_chunks/chunk_11.csv
Saved data_chunks/chunk_12.csv
Saved data_chunks/chunk_13.csv
Saved data_chunks/chunk_14.csv
Saved data_chunks/chunk_15.csv
Saved data_chunks/chunk_16.csv
Saved data_chunks/chunk_17.csv
Saved data_chunks/chunk_18.csv
Saved data_chunks/chunk_19.csv
Saved data_chunks/chunk_20.csv
Saved data_chunks/chunk_21.csv
Saved data_chunks/chunk_22.csv
Saved data_chunks/chunk_23.csv
Saved data_chunks/chunk_24.csv
Saved data_chunks/chunk_25.csv
Saved data_chunks/chunk_26.csv
Saved data_chunks/chunk_27.csv
Saved data_chunks/chunk_28.csv
Saved data_chunks/chunk_29.csv
Saved data_chunks/chunk_30.csv
Saved data_chunks/chunk_31.csv
Saved data_chunks/chunk_32.csv
Saved data_chunks

In [ ]:
cache_file_name="cache/tokenized_dataset.arrow"


In [ ]:
import os
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

# Path to your folder containing chunk CSV files
chunk_dir = "data_chunks"
chunk_files = sorted([os.path.join(chunk_dir, f) for f in os.listdir(chunk_dir) if f.endswith(".csv")])
print(f"Found {len(chunk_files)} chunk files.")

# Load base model and tokenizer once
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Set training parameters
use_fp16 = torch.cuda.is_available()
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=16,
    evaluation_strategy="no",
    save_strategy="no",
    logging_steps=100,
    fp16=use_fp16,
)

# Define functions outside the loop to avoid redefinition overhead
def map_labels(example):
    # Assuming CSV column 'labels' contains "Positive" or "Negative"
    example["label"] = 1 if example["labels"].strip().lower() == "positive" else 0
    return example

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

# Loop over each chunk file for incremental training
for i, chunk_file in enumerate(chunk_files, start=1):
    print(f"\n--- Processing chunk {i}/{len(chunk_files)}: {chunk_file} ---")

    # Load current chunk using Hugging Face datasets
    dataset = load_dataset("csv", data_files={"train": chunk_file})["train"]

    # Map string labels to integers
    dataset = dataset.map(map_labels)

    # Tokenize the dataset in batches
    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

    # Create Trainer and train on the current chunk
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
    )

    print("Starting training on current chunk...")
    trainer.train()
    print("Finished training on current chunk.")

    # Save a checkpoint after training on the current chunk
    ckpt_dir = os.path.join("checkpoints", f"chunk_{i}_checkpoint")
    os.makedirs(ckpt_dir, exist_ok=True)
    model.save_pretrained(ckpt_dir)
    tokenizer.save_pretrained(ckpt_dir)
    print(f"Checkpoint saved to {ckpt_dir}")

    # Clean up memory to reduce risk of kernel crashes
    del dataset, tokenized_dataset, trainer
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

print("\nIncremental training on all chunks is complete.")


2025-02-21 16:04:19.765302: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 37 chunk files.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/anaconda3/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(



--- Processing chunk 1/37: data_chunks/chunk_1.csv ---


Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Starting training on current chunk...


Step,Training Loss


In [ ]:
train_data_updated = '/Users/mac/SENTIMENT ANALYSIS PROJECT/train_dataset_updated'
train_df_updated = pd.read_csv(train_data_updated)